In [221]:
import pandas as pd
from datetime import date
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt

    # 01 - Campaign Data :(getting cam_duration)

In [293]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
def scale_df(data,cols):
    data_scaler = MinMaxScaler(feature_range=(0,1))
    scaling_cols = cols
    data_scaled = data[scaling_cols]
    data_scaled = data_scaler.fit_transform(data_scaled)
    data_scaled = pd.DataFrame(data_scaled,columns = scaling_cols)
    data = data.drop(scaling_cols,axis=1)
    data[scaling_cols] = data_scaled
    return data

In [2]:
cam_data = pd.read_csv('campaign_data.csv')

In [3]:
cam_data['end_date'] = pd.to_datetime(cam_data['end_date'],format='%d/%m/%y')
cam_data['start_date'] = pd.to_datetime(cam_data['start_date'],format='%d/%m/%y')

In [73]:
cam_data['cam_duration'] = (cam_data['end_date'] - cam_data['start_date']).dt.days
cam_data['campaign_type'] = cam_data['campaign_type'].replace('Y',0)
cam_data['campaign_type'] = cam_data['campaign_type'].replace('X',1)
cam_data.head()

,campaign_id,campaign_type,start_date,end_date,cam_duration
0,24,0,2013-10-21,2013-12-20,60
1,25,0,2013-10-21,2013-11-22,32
2,20,0,2013-09-07,2013-11-16,70
3,23,0,2013-10-08,2013-11-15,38
4,21,0,2013-09-16,2013-10-18,32


In [51]:
col = ['campaign_id','campaign_type','cam_duration']
#cam_data

    # 02 - coupon_item_mapping: (getting item_count for coupons)

In [5]:
coup_data = pd.read_csv('coupon_item_mapping.csv')

In [7]:
temp = coup_data.groupby(by = 'coupon_id').agg({'item_id':['count']})

In [8]:
coup_data2 = pd.DataFrame()
coup_data2['coupon_id'] = list(temp.iloc[0:,0].index)
coup_data2['item_count'] = list(temp.iloc[0:,0].values)

In [9]:
coup_data2.head()

,coupon_id,item_count
0,1,39
1,2,2
2,3,17
3,4,24
4,5,7


    # 03 - getting coupon count for items:

In [10]:
temp = coup_data.groupby(by = 'item_id').agg({'coupon_id':['count']})

In [11]:
coup_data3 = pd.DataFrame()
coup_data3['item_id'] = list(temp.iloc[0:,0].index)
coup_data3['coup_count'] = list(temp.iloc[0:,0].values)

In [12]:
coup_data3.tail()

,item_id,coup_count
36284,74054,4
36285,74056,2
36286,74057,2
36287,74058,2
36288,74061,2


    # 04 - customer_demographics: 

In [13]:
cust_data = pd.read_csv('customer_demographics.csv')

In [14]:
cust_data['marital_status'] = cust_data['marital_status'].fillna(0)
cust_data['marital_status'] = cust_data['marital_status'].replace('Married',2)
cust_data['marital_status'] = cust_data['marital_status'].replace('Single',1)

In [15]:
# bucketing Age range manually:
cust_data['age_range'] = cust_data['age_range'].replace('18-25',1)
cust_data['age_range'] = cust_data['age_range'].replace('26-35',2)
cust_data['age_range'] = cust_data['age_range'].replace('36-45',3)
cust_data['age_range'] = cust_data['age_range'].replace('46-55',4)
cust_data['age_range'] = cust_data['age_range'].replace('56-70',5)
cust_data['age_range'] = cust_data['age_range'].replace('70+',6)

In [16]:
cust_data['family_size'] = cust_data['family_size'].replace('5+',5)
cust_data['no_of_children'] = cust_data['no_of_children'].replace('3+',3)
cust_data['no_of_children'] = cust_data['no_of_children'].fillna(0)

In [17]:
cust_data.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,6,2,0,2,0,4
1,6,4,2,0,2,0,5
2,7,2,0,0,3,1,3
3,8,2,0,0,4,2,6
4,10,4,1,0,1,0,5


    # 05 - item_data : preprocess and adding coupon count for each item.

In [18]:
item_data = pd.read_csv('item_data.csv')

In [19]:
item_data['brand_type'] = item_data['brand_type'].replace('Established',1)
item_data['brand_type'] = item_data['brand_type'].replace('Local',0)

In [20]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [21]:
item_data['category'] = le.fit_transform(item_data['category'])

In [22]:
item_data.head()

,item_id,brand,brand_type,category
0,1,1,1,6
1,2,1,1,8
2,3,56,0,1
3,4,56,0,6
4,5,56,0,6


In [23]:
item_data = item_data.merge(coup_data3,on='item_id',how='left')

In [25]:
item_data['coup_count'] = item_data['coup_count'].fillna(0)
item_data['coup_count'] = item_data['coup_count'].astype(np.int64)
item_data.head()

,item_id,brand,brand_type,category,coup_count
0,1,1,1,6,2
1,2,1,1,8,0
2,3,56,0,1,0
3,4,56,0,6,3
4,5,56,0,6,0


    # 06 - Transaction_data : for each customer:

In [26]:
tra_data = pd.read_csv('customer_transaction_data.csv')

In [27]:
tra_data.columns

Index(['date', 'customer_id', 'item_id', 'quantity', 'selling_price',
       'other_discount', 'coupon_discount'],
      dtype='object')

In [28]:
def tran_count(cust):
    return len(tra_data[tra_data['customer_id']==cust]['date'].unique())
def item_count(cust):
    return len(tra_data[tra_data['customer_id']==cust]['item_id'].unique())
def get_sp(cust):
    return int(sum(tra_data[tra_data['customer_id'] == cust]['selling_price']))
def o_dis(cust):
    return int((tra_data[tra_data['customer_id']==cust]['other_discount']).sum()) * -1
def coup_dis(cust):
    return int(tra_data[tra_data['customer_id']==cust]['coupon_discount'].sum()) * -1
def coup_count(cust):
    z = tra_data[tra_data['customer_id']==cust]
    return z[z['coupon_discount']!=0]['coupon_discount'].count()

In [29]:
tran_data = pd.DataFrame()
tran_data['customer_id'] = list(tra_data['customer_id'].unique())

In [30]:
tran_data['tran_count'] = tran_data['customer_id'].map(tran_count)

In [31]:
tran_data['item_count_unique'] = tran_data['customer_id'].map(item_count)

In [32]:
tran_data['total_sp'] = tran_data['customer_id'].map(get_sp)

In [33]:
tran_data['total_o_dis'] = tran_data['customer_id'].map(o_dis)

In [34]:
tran_data['total_coup_dis'] = tran_data['customer_id'].map(coup_dis)

In [36]:
tran_data['coup_count'] = tran_data['customer_id'].map(coup_count)

In [37]:
tran_data.head()

,customer_id,tran_count,item_count_unique,total_sp,total_o_dis,total_coup_dis,coup_count
0,1501,53,273,43706,6443,845,31
1,857,97,374,48784,7833,1189,26
2,67,65,293,27293,4957,0,0
3,751,97,540,89772,13428,10,1
4,679,52,469,157174,22744,0,0


    #=============================Combining All================================#

In [454]:
t_data = pd.read_csv('train.csv')

In [455]:
col = ['campaign_id','campaign_type','cam_duration']
train_data = t_data.merge(cam_data[col],on='campaign_id',how='inner')
train_data = train_data.merge(coup_data2,on='coupon_id',how='inner')
train_data = train_data.merge(cust_data,on='customer_id',how='inner')
train_data['family_size'] = train_data['family_size'].astype(int)
train_data = train_data.merge(tran_data,on='customer_id',how='inner')

In [473]:
train_data['redemption_status'].value_counts()

0    43093
1      568
Name: redemption_status, dtype: int64

In [457]:
#cols = ['campaign_type', 'cam_duration', 'item_count', 'age_range','marital_status', 'rented', 'family_size', 'no_of_children',
#       'income_bracket', 'tran_count', 'item_count_unique', 'total_sp','total_o_dis', 'total_coup_dis', 'coup_count']
cols = ['campaign_type', 'cam_duration', 'item_count', 'age_range','marital_status', 'no_of_children',
       'income_bracket', 'tran_count', 'item_count_unique', 'total_coup_dis', 'coup_count']
X = train_data[cols]
y = train_data['redemption_status']
#===============================================
X = np.asarray(X)
X[0:5]
y = np.asarray(y)
y [0:5]

array([0, 0, 0, 0, 0], dtype=int64)

In [337]:
#from sklearn import preprocessing
#df_train = preprocessing.StandardScaler().fit(df_train).transform(df_train)
#df_train.head()
#=====================================================================================#
#X = scale_df(df_train,cols)
#X.shape

In [458]:
X = scale_df(df_train,cols)
X.shape

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


(43661, 15)

In [493]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)
print('0: ',np.unique(y_train,return_counts=True)[1][0],'\n1: ',np.unique(y_train,return_counts=True)[1][1])


Train set: (32745, 15) (32745,)
Test set: (10916, 15) (10916,)
0:  32308 
1:  437


# Model LR

In [460]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression()
LR.fit(X_train,y_train)

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [461]:
y_pred= LR.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [462]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print('Accuracy(test Data): ',accuracy_score(y_test, y_pred))
print('Accuracy(train Data):',accuracy_score(y_train,LR.predict(X_train)))
print('Confusion Matrix: \n',pd.DataFrame(confusion_matrix(y_test, y_pred)))
print('Classification Repost:\n',classification_report(y_test, y_pred))
#pd.DataFrame(classification_report(y_test, y_pred,output_dict=True))[['0','1']]

Accuracy(test Data):  0.9879992671308171
Accuracy(train Data): 0.9866544510612307
Confusion Matrix: 
        0  1
0  10785  0
1    131  0
Classification Repost:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99     10785
           1       0.00      0.00      0.00       131

   micro avg       0.99      0.99      0.99     10916
   macro avg       0.49      0.50      0.50     10916
weighted avg       0.98      0.99      0.98     10916



C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


    # XGB Classifier

In [463]:
import xgboost as xgb
xg = xgb.XGBClassifier(random_state=42,max_depth=9)
xg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=9, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [464]:
y_pred= xg.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [465]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print('Accuracy(test Data): ',accuracy_score(y_test, y_pred))
print('Accuracy(train Data):',accuracy_score(y_train,xg.predict(X_train)))
print('Confusion Matrix: \n',pd.DataFrame(confusion_matrix(y_test, y_pred)))
print('Classification Repost:\n',classification_report(y_test, y_pred))
#pd.DataFrame(classification_report(y_test, y_pred,output_dict=True))[['0','1']]

Accuracy(test Data):  0.9883657017222426
Accuracy(train Data): 0.9910826080317606
Confusion Matrix: 
        0   1
0  10773  12
1    115  16
Classification Repost:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99     10785
           1       0.57      0.12      0.20       131

   micro avg       0.99      0.99      0.99     10916
   macro avg       0.78      0.56      0.60     10916
weighted avg       0.98      0.99      0.98     10916

